# Part 1

Write a Python function called cky() that implements the CKY algorithm. This function should return True if words can be successfully parsed into a sentence and False otherwise.

In [1]:

def cky(words, grammar):

  #initiate the 5*6 table
  table = [[[] for _ in range(len(words) + 1)] for _ in range(len(words))]

  # outer loop
  for j in range(1, len(words) + 1):
    for key, value in grammar.items():
      if words[j-1] in value:
        table[j-1][j].append(key)

        # check (i,k) and (k,j) to fill (i,j)
        for i in range(j-2, -1, -1):
            for k in range(i+1, j):
              for a in table[i][k]:
                for b in table[k][j]:
                  combination = []
                  combination.append(a)
                  combination.append(b)
                  for key, value in grammar.items():
                    if combination in value:
                      table[i][j].append(key)

  # check (0,j)
  if 'S' in table[0][j]:
    return True
  else:
    return False


  ### OUTPUT CHECK ###
  # return True is words can be parsed into a sentence and False otherwise
  #return 'this is just a placeholder'
words = []
words = 'the cat tolerate a dog'.split()#sample, input any words string here##
grammar = {"NP" : [["D", "N"], "cats", "dogs"],
           "VP" : [["V", "NP"], "walk", "sleep"],
           "S"  : [["NP", "VP"]],
           "D"  : ["the", "a"],
           "N"  : ["cat", "dog"],
           "V"  : ["love", "tolerate"]
           }#sample, input any grammar here
cky(words,grammar)

True

Parse the a large CFG (rules.txt) into a dictionary.

In [114]:
def cfg(lines):
    grammar = {}
    for line in lines:
        parts = line.split("➡️")
        key = parts[0].strip()
        if "🦖" in parts[1]:
          value = [part.strip() for part in parts[1].split("🦖")]
        else:
          value = parts[1].strip()

        if key in grammar:
          if value not in grammar[key]:
            grammar[key].append(value)
        else:
            grammar[key] = [value]

    return grammar

cfg(open('rules.txt'))


{'S': [['NP', 'S|<VP-.>'],
  ['S', 'S|<,-NP-VP-.>'],
  ['NP', 'VP'],
  ['SBAR', 'S|<,-NP-VP-.>'],
  ['NP', "S|<VP-.-''>"],
  ['PP', 'S|<,-NP-VP>'],
  ['PP', 'S|<,-NP-VP-.>'],
  ['NP', 'S|<ADVP-VP>'],
  ['``', 'S|<CC-NP-VP-.>'],
  ['ADVP', 'S|<PRN-NP-``-VP-.>'],
  ['NP', 'S|<,-PP-,-VP-.>'],
  ['ADVP', 'S|<,-NP-VP-.>'],
  ['S', 'S|<:-S-.>'],
  ['SBAR', 'S|<,-NP-VP>'],
  ['S', 'S|<,-CC-S-.>'],
  ['PP', 'S|<NP-VP-.>'],
  ['SBAR', 'S|<,-NP-ADVP-VP-.>'],
  ['CC', 'S|<NP-VP-.>'],
  ['NP', 'S|<,-PP-,-NP-VP-.>'],
  ['S', 'S|<,-IN-S-.>'],
  ['ADVP', 'S|<,-PP-,-NP-VP-.>'],
  ['S+NP', 'S|<,-NP-VP-.>'],
  ['NP', 'S|<,-NP-VP-.>'],
  ['S+VP', 'S|<,-NP-VP-.>'],
  ['CC', 'S|<PP-,-NP-VP-.>'],
  ['NP', 'S|<,-S-,-VP-.>'],
  ['S', 'S|<:-S-:-S-.>'],
  ['PP', 'VP'],
  ['S', 'S|<,-CC-S>'],
  ['SBAR', 'VP'],
  ['PP', 'S|<NP-VP>'],
  ['S', 'S|<CC-S-S>'],
  ['NP', 'ADVP'],
  ['ADVP', 'VP'],
  ['S', 'S|<CC-S-.>'],
  ['S', 'S|<CC-S>'],
  ['``', 'S|<SBAR-,-S-,-CC-S-.>'],
  ['S', "S|<,-''-NP-VP-.>"],
  ['NP', 'S|<AD

# Part 2

A function called uniform() that takes a CFG stored as a dictionary and assigns uniform probabilities to all its rules. Do this by replacing the expansions with
lists containing the expansion as the first element and its probability as the second.

In [95]:
def uniform(grammar):
  for key, value in grammar.items():
        if isinstance(value, list):
            total_expansions = len(value)
            probability = 1.0 / total_expansions
            for i, item in enumerate(value):
                    grammar[key][i] = [item, probability]

# grammar
grammar = {
    "NP": [["D", "N"], "cats", "dogs"],
    "VP": [["V", "NP"], "walk", "sleep"],
    "S": [["NP", "VP"]],
    "D": ["the", "a"],
    "N": ["cat", "dog"],
    "V": ["love", "tolerate"]
    }

  ### OUTPUT CHECK###
uniform(grammar)
grammar

{'NP': [[['D', 'N'], 0.3333333333333333],
  ['cats', 0.3333333333333333],
  ['dogs', 0.3333333333333333]],
 'VP': [[['V', 'NP'], 0.3333333333333333],
  ['walk', 0.3333333333333333],
  ['sleep', 0.3333333333333333]],
 'S': [[['NP', 'VP'], 1.0]],
 'D': [['the', 0.5], ['a', 0.5]],
 'N': [['cat', 0.5], ['dog', 0.5]],
 'V': [['love', 0.5], ['tolerate', 0.5]]}

A function called pcky() that implements a probabilistic version of CKY algorithm. It takes two arguments: words, a list of strings representing a sequence of words, and grammar, a probabilistic CFG stored as a dictionary. 

In [131]:
import math

def pcky(words, grammar):
	#initiate the 5*6 table
  table = [[[] for _ in range(len(words) + 1)] for _ in range(len(words))]

  for j in range(1, len(words) + 1):
        for key, value in grammar.items():
            for c in value:
                if words[j - 1] == c[0]:
                    table[j - 1][j].append({key: math.log(c[1])})


        for i in range(j - 2, -1, -1):
            for k in range(i + 1, j):
                for d in table[i][k]:
                    for key_1, value_1 in d.items():
                        a = key_1
                        p_ik = value_1
                    for e in table[k][j]:
                        for key_2, value_2 in e.items():
                            b = key_2
                            p_kj = value_2
                            combination = []
                            combination.append(a)
                            combination.append(b)
                            p = p_ik + p_kj

                            for key_3, value_3 in grammar.items():
                                for f in value_3:
                                    if combination == f[0]:
                                        table[i][j].append({key_3: math.log(f[1]) + p})


  tell_S = False
  max_value = None
  for h in table[0][j]:
        if 'S' in h:
            if not tell_S:
                tell_S = True
                max_value = h['S']
            else:
                if h['S'] >= max_value:
                    max_value = h['S']

  if not tell_S:
        return False
  else:
        return max_value

### OUTPUT CHECK###
#words
words = ['the', 'cat', 'tolerate', 'a', 'dog']

###dictionary
grammar = {
    "NP": [["D", "N"], "cats", "dogs"],
    "VP": [["V", "NP"], "walk", "sleep"],#
    "S": [["NP", "VP"]],
    "D": ["the", "a"],
    "N": ["cat", "dog"],
    "V": ["love", "tolerate"]
}
uniform(grammar)
print(pcky(words, grammar))



-6.761572768804056


# Part 3

Use pos files to estimate the parameters of an HMM.

In [140]:
# wsj_0XXX.pos files are available in tagged.zip
A = {}
B = {}
Π = {}

import collections
tag_list = []
word_list = []
lines_string = ""
all_tags = set()


file_paths = ["wsj_{:04d}.pos".format(i) for i in range(1, 200)]

for file_path in file_paths:
    prev_tag = ""
    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()
            line = line.replace("[", "").replace("]", "")
            if line.startswith("=") or not line:
                continue

            lines_string += line + " " 
        print(lines_string)


# parse the line
lines_string = lines_string.replace("./.", "./.😊")
lines = lines_string.split("😊")


for line in lines:
    if not line.strip():
        continue
    tokens = line.split()

    for i, token in enumerate(tokens):
        if "/" not in token:
            continue
        word, tag = token.rsplit("/", 1)
        tag_list.append(tag)
        word_list.append(word)
        all_tags.add(tag)


        # first word in the sentence
        if i == 0:
            if tag in Π:
                Π[tag] += 1
            else:
                Π[tag] = 1
        else:
            # transition
            if prev_tag not in A:
                A[prev_tag] = {}
            if tag not in A[prev_tag]:
                A[prev_tag][tag] = 0
            A[prev_tag][tag] += 1

        # emission
        if tag not in B:
            B[tag] = {}
        if word not in B[tag]:
            B[tag][word] = 0
        B[tag][word] += 1

        prev_tag = tag

# Π
# Ensure Π includes all tags and normalize
for tag in all_tags:
    if tag not in Π:
        Π[tag] = 0

total = sum(Π.values())
if total != 0:
    for tag in Π:
        Π[tag] /= total
else:
    for tag in Π:
        Π[tag] = 0
total = sum(Π.values())
if total != 0:
    for tag in Π:
        Π[tag] /= total

# A
# Ensure A includes all tags and normalize
for tag in all_tags:
    if tag not in A:
        A[tag] = {}
    for tag_2 in all_tags:
      if tag_2 not in A[tag]:
        A[tag][tag_2] = 0

for a in A:
    total = sum(A[a].values())
    if total != 0:
      for b in A[a]:
        A[a][b] /= total
    else:
      for b in A[a]:
        A[a][b] = 0

# B
for tag in all_tags:
    if tag not in B:
        B[tag] = {}
    for word_2 in word_list:
      if word_2 not in B[tag]:
        B[tag][word_2] = 0

for q in B:
    total = sum(B[q].values())
    if total != 0:
      for o in B[q]:
        B[q][o] /= total
    else:
      for o in B[q]:
        B[q][o] = 0

# Print the results (optional)
print("Π:", Π)
print("A:", A)
print("B:", B)

#sanity check
tag_counts = collections.Counter(tag_list)
unique_tags = len(tag_counts)
word_counts = collections.Counter(word_list)
unique_words = len(word_counts)
print("-------sanity check-------")
print("tag_counts:", unique_tags)
print("word_counts:",unique_words)








IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Pick ten random sentences from the file sentences.txt and compare the probabilities assigned by the probabilistic CKY function to those of the Viterbi algorithm.

In [137]:
import math

def viterbi(A, B, Π, words):
  lattice = []
  for o in words:
      lattice.append({})
      for q in A:
        lattice[-1][q] = [float(0),'']

  # initialization step
  for q in A:
      if q in Π and words[0] in B[q]:
          lattice[0][q][0] = Π[q] * B[q][words[0]]

  # recursive step
  for t in range(1, len(words)):
      for qⱼ in A:
        max = float('0')
        pre = ''
        for qᵢ in A:
          p = lattice[t-1][qᵢ][0] * A[qᵢ][qⱼ] * B[qⱼ][words[t]]
          if p > max:
            max = p
            pre = qᵢ
        lattice[t][qⱼ] = [max, pre]


  path = []
  # find most prob state at time T
  max = float('-inf')
  end = ''
  for q in A:
      if lattice[-1][q][0] > max:
        max = lattice[-1][q][0]
        end = q

  path.append(end)
  backpoint = path[-1]
  for t in range(len(words) - 1, 0, -1):
      path.append(lattice[t][backpoint][1])
      backpoint = path[-1]

  return math.log(max)

### PCKY ###
### cfg grammar


grammar = cfg(open('rules.txt'))
uniform(grammar)


  ### OUTPUT ###
  # return the log likelihood of the most probable sequence of hidden states
  #return 'this is just a placeholder'
print("----------------viterbi output----------------")
sentences = [
    "Champagne and dessert followed .",
    "Pick a country , any country .",
    "The U.S.S.R. belongs to neither organization .",
    "Her alternative was 90 days in jail .",
    "South Korea has different concerns .",
    "Cathryn Rice could hardly believe her eyes .",
    "I believe in the system .",
    "`` I draw a blank . ''",
    "All came from Cray Research .",
    "`` That attracts attention ..."
]
for sentence in sentences:
    words = sentence.split()
    viterbi_log_prob = viterbi(A, B, Π, words)
    print(f"Sentence: {sentence}")
    print(f"Viterbi Log Probability: {viterbi_log_prob}")

print("------------------PCKY output------------------")
#words = "Champagne and dessert followed .".split()
#pcky(words, grammar)
for sentence in sentences:
    words = sentence.split()
    cky_results = pcky(words, grammar)
    print(f"Sentence: {sentence}")
    print(f"CKY Log Probability: {cky_results}")



----------------viterbi output----------------
Sentence: Champagne and dessert followed .
Viterbi Log Probability: -37.21512016749909
Sentence: Pick a country , any country .
Viterbi Log Probability: -39.81856369746412
Sentence: The U.S.S.R. belongs to neither organization .
Viterbi Log Probability: -47.64210688061698
Sentence: Her alternative was 90 days in jail .
Viterbi Log Probability: -55.4426928761881
Sentence: South Korea has different concerns .
Viterbi Log Probability: -37.462421004740975
Sentence: Cathryn Rice could hardly believe her eyes .
Viterbi Log Probability: -61.70530527532861
Sentence: I believe in the system .
Viterbi Log Probability: -27.693944977995766
Sentence: `` I draw a blank . ''
Viterbi Log Probability: -33.24627894041153
Sentence: All came from Cray Research .
Viterbi Log Probability: -41.350183824060004
Sentence: `` That attracts attention ...
Viterbi Log Probability: -40.934838773238155
------------------PCKY output------------------
Sentence: Champagne a

viterbi output：

Sentence: Champagne and dessert followed .

Viterbi Log Probability: -37.21512016749909

Sentence: Pick a country , any country .

Viterbi Log Probability: -39.81856369746412

Sentence: The next province ?

Viterbi Log Probability: -27.26323409053319

Sentence: It 's imaginative and often funny .

Viterbi Log Probability: -47.6282461439437

Sentence: South Korea has different concerns .

Viterbi Log Probability: -37.462421004740975

Sentence: This is Japan ?

Viterbi Log Probability: -28.254575976046777

Sentence: Who 's telling the truth ?

Viterbi Log Probability: -42.08550907397408

Sentence: He was previously vice president .

Viterbi Log Probability: -36.518522431980415

Sentence: All came from Cray Research .

Viterbi Log Probability: -41.350183824060004

Sentence: He is his own man .

Viterbi Log Probability: -31.55667294386035

------------------PCKY output------------------

Sentence: Champagne and dessert followed .

CKY Log Probability: -43.275273025735025

Sentence: Pick a country , any country .

CKY Log Probability: -59.061454615414775

Sentence: The U.S.S.R. belongs to neither organization .

CKY Log Probability: -59.3251969022203

Sentence: Her alternative was 90 days in jail .

CKY Log Probability: -74.00153374106864

Sentence: South Korea has different concerns .

CKY Log Probability: -60.93992164375324

Sentence: Cathryn Rice could hardly believe her eyes .

CKY Log Probability: -75.57128889992995

Sentence: I believe in the system .

CKY Log Probability: -56.440245630308496

Sentence: `` I draw a blank . ''

CKY Log Probability: -53.07408474539902

Sentence: All came from Cray Research .

CKY Log Probability: -56.67553279802686

Sentence: `` That attracts attention ...

CKY Log Probability: -44.22445435844077


Results and Discussions:

The results show that all sentences score higher likelihoods on Viterbi than PCKY.

Properties Accounting for Differences between CKY and Viterbi (HMM):

Using CKY and PCKY algorithm to obtain dependency parses of phrases/sentences, which is only possible if they obey given CFG rules. As a result, PCKY can capture complex syntactic dependencies better due to its use of a detailed probabilistic CFG derived from actual reasonable rules. However, it is greatly influenced by factors such as the shape of the trees, and labels of the tree. This typically leads to a more fine-grained partitioning of the training data. But it is very slow.

Viterbi (HMM) deals with initial tags, tag/tag, and tag/word dependencies through state transitions and emissions. It is no necessary to think about syntax, but just focuses on the changing pattern of any two adjacent components. It yields higher log probabilities for sentences with straightforward dependencies.

Conclusion：
The probabilistic CKY algorithm is fine-grained but was outperformed by the Viterbi algorithm in time due to the latter’s efficiency in managing sequential data and local dependencies,

Use the file rules.txt to estimate the parameters of the probabilistic CFG. Compare the probabilities assigned by your probabilistic CKY function using this new grammar to those of the old grammar and those of the Viterbi algorithm.

In [145]:
new_grammar = {}

with open("rules.txt") as f:
    lhs_count = {}
    for line in f:
        line = line.strip()
        linespl = line.split("➡️")
        linespl[0] = linespl[0].strip()
        if "🦖" in linespl[1]:
            linespl[1] = [part.strip() for part in linespl[1].split("🦖")]
        else:
            linespl[1] = linespl[1].strip()

        a = linespl[0]
        b = linespl[1]

        if a not in lhs_count:
            lhs_count[a] = 0
        lhs_count[a] += 1

        if a not in new_grammar:
            new_grammar[a] = [[b, 1]]
        else:
            found = False
            for item in new_grammar[a]:
                if item[0] == b:
                    item[1] += 1
                    found = True
                    break
            if not found:
                new_grammar[a].append([b, 1])

for a in new_grammar:
    total = sum(item[1] for item in new_grammar[a])
    for item in new_grammar[a]:
        item[1] = item[1] / total

# Probabilistic CKY algorithm

sentences = [
    "Champagne and dessert followed .",
    "Pick a country , any country .",
    "The U.S.S.R. belongs to neither organization .",
    "Her alternative was 90 days in jail .",
    "South Korea has different concerns .",
    "Cathryn Rice could hardly believe her eyes .",
    "I believe in the system .",
    "`` I draw a blank . ''",
    "All came from Cray Research .",
    "`` That attracts attention ..."
]
for sentence in sentences:
    words = sentence.split()
    cky_results = pcky(words, new_grammar)
    print(f"Sentence: {sentence}")
    print(f"CKY Log Probability: {cky_results}")




{'S': [[['NP', 'S|<VP-.>'], 0.2742342911272498], [['S', 'S|<,-NP-VP-.>'], 0.017998105462582886], [['NP', 'VP'], 0.34101673508051783], [['SBAR', 'S|<,-NP-VP-.>'], 0.012946005683612252], [['NP', "S|<VP-.-''>"], 0.012472371329333754], [['PP', 'S|<,-NP-VP>'], 0.0058414903694347965], [['PP', 'S|<,-NP-VP-.>'], 0.04262709188506473], [['NP', 'S|<ADVP-VP>'], 0.013103883801705084], [['``', 'S|<CC-NP-VP-.>'], 0.00015787811809283233], [['ADVP', 'S|<PRN-NP-``-VP-.>'], 0.00015787811809283233], [['NP', 'S|<,-PP-,-VP-.>'], 0.0022102936532996525], [['ADVP', 'S|<,-NP-VP-.>'], 0.014998421218819072], [['S', 'S|<:-S-.>'], 0.006630880959898958], [['SBAR', 'S|<,-NP-VP>'], 0.005683612251341964], [['S', 'S|<,-CC-S-.>'], 0.01484054310072624], [['PP', 'S|<NP-VP-.>'], 0.0029996842437638145], [['SBAR', 'S|<,-NP-ADVP-VP-.>'], 0.000473634354278497], [['CC', 'S|<NP-VP-.>'], 0.01768234922639722], [['NP', 'S|<,-PP-,-NP-VP-.>'], 0.00031575623618566466], [['S', 'S|<,-IN-S-.>'], 0.00015787811809283233], [['ADVP', 'S|<,-PP

------------NEW_grammar PCKY results------------

Sentence: Champagne and dessert followed .

CKY Log Probability: -46.450120753484995

Sentence: Pick a country , any country .

CKY Log Probability: -43.32823271567303

Sentence: The U.S.S.R. belongs to neither organization .

CKY Log Probability: -48.85400389886128

Sentence: Her alternative was 90 days in jail .

CKY Log Probability: -59.45129216401506

Sentence: South Korea has different concerns .

CKY Log Probability: -36.94895088353458

Sentence: Cathryn Rice could hardly believe her eyes .

CKY Log Probability: -60.632716262781706

Sentence: I believe in the system .

CKY Log Probability: -45.843776759129376

Sentence: `` I draw a blank . ''

CKY Log Probability: -56.66603737007935

Sentence: All came from Cray Research .

CKY Log Probability: -50.64227371453571

Sentence: `` That attracts attention ...

CKY Log Probability: -54.25957020996016


Results and discussion:

Seven sentence, "Pick a country , any country .", "The U.S.S.R. belongs to neither organization .", "Her alternative was 90 days in jail .", "South Korea has different concerns .", "Cathryn Rice could hardly believe her eyes .", "I believe in the system .", "All came from Cray Research .", score a higher likelihood on new PCKY than old one. The new probabilistic grammar enhances probabilities compared to the old uniform probabilistic grammar. This improvement is due to the data-driven estimation of rule probabilities, which makes parsing more representative of actual language structures.

Comparison with HMM (Viterbi), HMM still offers competitive log probabilities, which is higher than those of the new probabilistic CFG. This consistency arises from the HMM's ability to handle transitions and emissions probabilistically at each step. The new probabilistic CFG occasionally produces higher log probabilities than the HMM, indicating that the CFG can capture syntactic dependencies that the HMM might miss, particularly in cases involving complex syntactic structures.




